In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from torchvision.datasets import MNIST

In [27]:
batch_size_train = 64
batch_size_test = 1000
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST("MNIST/processed/training.pt", train=True, download=False,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST("MNIST/processed/test.pt", train=False, download=False,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)
print(train_loader)

In [28]:
def reshape_data():
    for batch_id, (images, labels) in enumerate(train_loader):
        #used to flatten data to get shape of [64 by 784]
        images = images.view(64, -1)
        print(images.size())
    for batch_id, (images, labels) in enumerate(test_loader):
        images = images.view(64, -1)
#Note features are arranged in columns giving row vectors thus must y = X.W + b

In [43]:
class NeuralNetwork(nn.Module):
    def __init__(self, ):
        super(NeuralNetwork, self).__init__()
        self.nodes_layer_1 = 100
        self.nodes_layer_2 = 50
        self.input_layers = 784
        self.output_layers = 10
        self.training_batch_size = 64
        self.learning_rate = 0.05
        
        self.W1 = torch.randn(self.input_layers, self.nodes_layer_1)
        self.b1 = torch.zeros(self.training_batch_size, 100)
        
        self.W2 = torch.randn(self.nodes_layer_1, self.nodes_layer_2)
        self.b2 = torch.zeros(self.training_batch_size, 100)
        
        self.W3 = torch.randn(self.nodes_layer_2, self.output_layers)
        self.b3 = torch.zeros(self.training_batch_size, 10)
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1-s)
    
    def forward(self, x):
        self.z1 = self.sigmoid(torch.matmul(x, self.W1) + self.b1)
        self.z2 = self.sigmoid(torch.matmul(self.z1, self.W2) + self.b2)
        o = self.sigmoid(torch.matmul(self.z2, self.W3) + self.b3)
        return o
    
    def backward(self, x, y, o):
        loss = (o - y).pow(2)
        o_error = 2 * (o-y)
        #Note db3 is the same as dz3
        dz3 = o_error * self.sigmoidPrime(o)
        dW3 = torch.matmul(dz3, torch.t(self.z2))
        #Note db2 is the same as dz2
        dz2 = torch.matmul(dz2, torch.t(self.W3))*self.sigmoidPrime(self.z2)
        dW2 = dz2 * self.z1
        #Note db1 is the same as dz1
        dz1 = torch.matmul(dz2, torch.t(self.W2))*self.sigmoidPrime(self.z1)
        dW1 = dz1 * x
        
        self.W3 = self.W3 - (self.learning_rate*dW3)
        self.b3 = self.b3 - (self.learning_rate*dz3)
        
        self.W2 = self.W2 - (self.learning_rate*dW2)
        self.b2 = self.b2 - (self.learning_rate*dz2)
        
        self.W1 = self.W1 - (self.learning_rate*dW1)
        self.b1 = self.b1 - (self.learning_rate*dz1)
    
    def train(self, x, y):
        # forward + backward pass for training
        o = self.forward(x)
        self.backward(x, y, o)
    
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
#     def predict(self):
#         print ("Predicted data based on trained weights: ")
#         print ("Input (scaled): \n" + str(x_test))
#         print ("Output: \n" + str(self.forward(x_test)))

In [44]:
NN = NeuralNetwork()
epochs = len(train_loader)

for batch_id, (images, labels) in enumerate(train_loader):
    if images.size != torch.Size([64, 1, 28, 28]):
        continue
    x = images.view(64, -1)
    print ("#" + str(batch_id) + " Loss: " + str(torch.mean((y - NN(x))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)

/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type NeuralNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
